<a href="https://colab.research.google.com/github/Zenith1618/Learn-PySpark/blob/main/Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [2]:
df = spark.read.format('csv').option('header', 'true').load("challenge.csv")

In [3]:
df.dtypes

[('ip_address', 'string'),
 ('Country', 'string'),
 ('Domain Name', 'string'),
 ('Bytes_used', 'string')]

In [19]:
from pyspark.sql.types import *

myschema = StructType([
    StructField('ip_address', StringType()),
    StructField('Country', StringType()),
    StructField('Domain Name', StringType()),
    StructField('Bytes_used', IntegerType())
])

df = spark.read.csv("challenge.csv", header=True, schema=myschema)
df.show()

+---------------+--------------+-----------------+----------+
|     ip_address|       Country|      Domain Name|Bytes_used|
+---------------+--------------+-----------------+----------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|
| 119.239.207.13|         China|         youtu.be|        51|
|  68.69.217.210|         China|        adobe.com|        10|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|
|   211.13.10.68|     Indonesia|          hud.gov|        29|
|   239.80.21.97|      Suriname|       smh.com.au|       218|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|
| 127.242.24.138|         China| surveymonkey.com|       123|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|
|   237.54.11.63|         China|       amazon.com|        83|
| 252.141.157.25|         Japan|      cornell.edu|       374|
|185.220.128.248|       Belgium|       weebly.com|       389|
|   151.77.19.45|   Afghanistan|independent.co.uk|       282|
|  9.161

In [20]:
df.dtypes

[('ip_address', 'string'),
 ('Country', 'string'),
 ('Domain Name', 'string'),
 ('Bytes_used', 'int')]

In [21]:
from pyspark.sql.functions import *
df = df.withColumn('Mexico', when(df.Country == "Mexico", "YES").otherwise("NO"))

In [22]:
df.show()

+---------------+--------------+-----------------+----------+------+
|     ip_address|       Country|      Domain Name|Bytes_used|Mexico|
+---------------+--------------+-----------------+----------+------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|    NO|
| 119.239.207.13|         China|         youtu.be|        51|    NO|
|  68.69.217.210|         China|        adobe.com|        10|    NO|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|    NO|
|   211.13.10.68|     Indonesia|          hud.gov|        29|    NO|
|   239.80.21.97|      Suriname|       smh.com.au|       218|    NO|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|    NO|
| 127.242.24.138|         China| surveymonkey.com|       123|    NO|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|    NO|
|   237.54.11.63|         China|       amazon.com|        83|    NO|
| 252.141.157.25|         Japan|      cornell.edu|       374|    NO|
|185.220.128.248|       Belgium|  

In [23]:
import pyspark.sql.functions as sqlfunc

df1 = df.groupBy('Mexico').agg(sqlfunc.sum('Bytes_used').alias('total_used'))
df1.show()

+------+----------+
|Mexico|total_used|
+------+----------+
|   YES|      6293|
|    NO|    508076|
+------+----------+



In [25]:
df2 = df.groupBy('Country').agg(sqlfunc.countDistinct('ip_address').alias("number of IP's"))
df2.sort(col("number of IP's").desc()).show()

+--------------+--------------+
|       Country|number of IP's|
+--------------+--------------+
|         China|           172|
|     Indonesia|           114|
|   Philippines|            65|
|        Russia|            56|
|        Brazil|            35|
|        Poland|            31|
|        Sweden|            28|
|         Japan|            25|
|      Portugal|            23|
|Czech Republic|            23|
|        France|            21|
|          Peru|            19|
|      Colombia|            17|
| United States|            15|
|     Argentina|            14|
|       Ukraine|            14|
|        Mexico|            13|
|      Thailand|            12|
|       Nigeria|            11|
|        Canada|            11|
+--------------+--------------+
only showing top 20 rows

